In [175]:
import requests
import csv 
import pandas as pd
import os
from bs4 import BeautifulSoup

class crime_data:
    # constructor that generates the dictionary of the cities and its frequencies
    def __init__(self):
        self.dic = {'Seattle':0,"Bellevue":0,"Redmond":0,"Kirkland":0,"Bothell":0,"Tacoma":0,"Issaquah":0,"Everett":0,"Woodinville":0,
            "Bainbridge Island":0,"Mercer Island":0,"Sammamish":0,"Olympia":0,"Lynnwood":0,"Renton":0,"Snohomish":0,"Puyallup":0,
           "Kent":0,"Auburn":0,"Edmonds":0,"Kenmore":0,"Mount Vernon":0,"Mukilteo":0,"Lakewood":0,"Federal Way":0,"Gig Harbor":0,
           "Shoreline":0,"Snoqualmie":0,"Bremerton":0,"Arlington":0,"Poulsbo":0,"Lake Stevens":0,"Tukwila":0,"Milton":0,
           "Spanaway":0,"Tumwater":0,"Bonney Lake":0,"Marysville":0,"Silverdale":0,"Monroe":0,"Maple Valley":0,"Vashon":0,
           "North Bend":0,"University Place":0,"Mountlake Terrace":0,"Lacey":0,"Graham":0,"Anacortes":0,"Fall City":0,
           "Medina":0,"Port Orchard":0,"Black Diamond":0,"Lake Forest Park":0,"Enumclaw":0,"Stanwood":0,"Fox Island":0,
           "Lakebay":0,"Burlington":0,"Duvall":0,"Seahurst":0,"Southworth":0,"Gold Bar":0,"Tracyton":0,"South Colby":0,
           "Concrete":0,"Steilacoom":0,"Carnation":0,"Yelm":0,"Sumner":0,"Ashford":0,"RochesterOrting":0,"Buckley":0,
           "Seabeck":0,"Fife":0,"Startup":0,"Rockport":0,"Littlerock":0,"Preston":0,"Hamilton":0,"Silvana":0,"Longbranch":0,
           "Manchester":0,"Bucoda":0,"Burley":0,"Wauna":0,"Keyport":0,"Port Gamble":0,"South Prairie":0,"Roy":0,
           "Suquamish":0,"Skykomish":0,"Pacific":0,"Conway":0,"Burton":0,"Marblemount":0,"Lyman":0,"Hobart":0,"Sultan":0,
           "Indianola":0,"Eatonville":0,"Index":0,"Darrington":0,"Hansville":0,"Camp Murray":0,"Carbonado":0,"Baring":0,
           "Tenino":0,"Granite Falls":0,"Kingston":0,"Elbe":0,"Longmire":0,"Bow":0,"Rollingbay":0,"Kapowsin":0,"Vaughn":0,
           "La Conner":0,"Rainier":0,"Wilkeson":0,"Paradise Inn":0,"East Olympia":0,"Retsil":0,"Olalla":0,"Ravensdale":0,
           "La Grande":0,"Redondo":0}
        try: #if data.csv is found then take the frequencies in to dictionary           
            with open('data.csv','r') as csvfile:
                reader= csv.reader(csvfile)
                mydict = {rows[0][0:]:rows[1][0:] for rows in reader}
                mydict.pop('region')
                for key in self.dic:                
                    self.dic[key]=int(mydict[key])
                    
        except : #if data.csv is not found, create data.csv and store the value into csv file
            header = ['region', 'frequency']
            with open('data.csv','w+') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(header)
                for key, value in self.dic.items():
                    writer.writerow([key, value])
                    
        try: #check if the the crime stats.txt exist 
            with open('used_crime_stats.txt','r',encoding='utf-8') as yfile:
                print()
                
                
        except FileNotFoundError:#if the file don't exist created one
            with open('used_crime_stats.txt','w',encoding='utf-8') as yfile:
                yfile.write(' ')
                yfile.close()
                    
                    
    def compare(self): #compare the text from the news and increased the frequencies in the dictionary 
        with open('crime_stats.txt','r',encoding='utf-8') as file,open('used_crime_stats.txt','w',encoding='utf-8')as yfile:
            for line in file:
                words = line.split()
                for city in self.dic:
                    if city in words:
                        self.dic[city]+=1
            #reset the pointer to the first line in the file to write it as used news
            file.seek(0)
            for lines in file:
                yfile.write(lines)
            #write it into data.csv so we can easily use pandas to print    
            with open('data.csv','w') as csvfile:
                header = ['region', 'frequency']
                writer = csv.writer(csvfile)
                writer.writerow(header)
                for key, value in self.dic.items():
                    writer.writerow([key, value])

    #request the content in the website and skip the lines that always on the news
    def get(self):
        with open('crime_stats.txt','w+',encoding='utf-8') as file,open('used_crime_stats.txt','r',encoding='utf-8') as yfile:
                i = 0
                r = requests.get("https://www.seattlepi.com/news/crime/")
                soup = BeautifulSoup(r.text,"html.parser") 
                writer = csv.writer(file)
                for a in soup.find_all('a', attrs={'class':'hdn-analytics'}):
                    i+=1
                    if i>71:
                        file.write(a.get_text())
    #start method is used to connect all the method together and 
    #if the news hasn't been used it will add it and marked it as used
    def start(self):
        self.get()
        if(self.check()):
            self.compare()
            with open('crime_stats.txt','w+',encoding='utf-8') as file,open('used_crime_stats.txt','r',encoding='utf-8') as yfile:
                file.seek(0)
                flines = file.readlines()
                for line in flines:
                    yfile.write(line)              
         
            
    #check is used to read crime stats and used crime stats and compare these two, if they are identical then return false
    #else returm true
    def check(self):
        count= 0
        with open('crime_stats.txt','r',encoding='utf-8') as file,open('used_crime_stats.txt','r',encoding='utf-8') as yfile:
            flines = file.readlines()
            ylines = yfile.readlines()
            if ylines[0]==' ':
                print('sucessfuly added count')
                return True
            try:
                for line in flines:
                    if(line==ylines[count]):
                        print('This news is already added')
                        return False
                    else:
                        count+=1
            except IndexError:       
                print('sucessfuly added count')        
                return True
    #it print the data.csv into console
    def show(self):
        try:
            df = pd.read_csv('data.csv')
            print(df)
            print(df.sort_values(by='frequency',ascending=False)) 
        except FileNotFoundError:
            print('the data is cleaned or empty')
    #clean metho is used to delete all the files        
    def clean(self):
        print('clean clled')
        self.dic = {'Seattle':0,"Bellevue":0,"Redmond":0,"Kirkland":0,"Bothell":0,"Tacoma":0,"Issaquah":0,"Everett":0,"Woodinville":0,
            "Bainbridge Island":0,"Mercer Island":0,"Sammamish":0,"Olympia":0,"Lynnwood":0,"Renton":0,"Snohomish":0,"Puyallup":0,
           "Kent":0,"Auburn":0,"Edmonds":0,"Kenmore":0,"Mount Vernon":0,"Mukilteo":0,"Lakewood":0,"Federal Way":0,"Gig Harbor":0,
           "Shoreline":0,"Snoqualmie":0,"Bremerton":0,"Arlington":0,"Poulsbo":0,"Lake Stevens":0,"Tukwila":0,"Milton":0,
           "Spanaway":0,"Tumwater":0,"Bonney Lake":0,"Marysville":0,"Silverdale":0,"Monroe":0,"Maple Valley":0,"Vashon":0,
           "North Bend":0,"University Place":0,"Mountlake Terrace":0,"Lacey":0,"Graham":0,"Anacortes":0,"Fall City":0,
           "Medina":0,"Port Orchard":0,"Black Diamond":0,"Lake Forest Park":0,"Enumclaw":0,"Stanwood":0,"Fox Island":0,
           "Lakebay":0,"Burlington":0,"Duvall":0,"Seahurst":0,"Southworth":0,"Gold Bar":0,"Tracyton":0,"South Colby":0,
           "Concrete":0,"Steilacoom":0,"Carnation":0,"Yelm":0,"Sumner":0,"Ashford":0,"RochesterOrting":0,"Buckley":0,
           "Seabeck":0,"Fife":0,"Startup":0,"Rockport":0,"Littlerock":0,"Preston":0,"Hamilton":0,"Silvana":0,"Longbranch":0,
           "Manchester":0,"Bucoda":0,"Burley":0,"Wauna":0,"Keyport":0,"Port Gamble":0,"South Prairie":0,"Roy":0,
           "Suquamish":0,"Skykomish":0,"Pacific":0,"Conway":0,"Burton":0,"Marblemount":0,"Lyman":0,"Hobart":0,"Sultan":0,
           "Indianola":0,"Eatonville":0,"Index":0,"Darrington":0,"Hansville":0,"Camp Murray":0,"Carbonado":0,"Baring":0,
           "Tenino":0,"Granite Falls":0,"Kingston":0,"Elbe":0,"Longmire":0,"Bow":0,"Rollingbay":0,"Kapowsin":0,"Vaughn":0,
           "La Conner":0,"Rainier":0,"Wilkeson":0,"Paradise Inn":0,"East Olympia":0,"Retsil":0,"Olalla":0,"Ravensdale":0,
           "La Grande":0,"Redondo":0}
        with open('crime_stats.txt','w',encoding='utf-8') as file,open('used_crime_stats.txt','w',encoding='utf-8') as yfile:
                with open('data.csv','w') as csvfile:
                    os.remove('crime_stats.txt')
                    os.remove("used_crime_stats.txt")
                    os.remove("data.csv")
                    print('cleaned!')
            

In [ ]:
#main method
if __name__ == "__main__":
    object1 = crime_data()
    userinput=-1
    object1.show()
    try:
        while (userinput!= '0'):
            print('enter 1 to calculate the count of sities showing up onthe crime news web page')
            print('enter 2 to show the data on console')
            print('enter 3 to clean the data')
            print('enter 0 to exit')
            userinput=input('enter your input \n')
            if(userinput=='1'):
                object1.start()
            elif(userinput=='2'):
                object1.show()
            elif(userinput=='3'):
                object1.clean()
                object1=crime_data()
            else:
                print('invalid input')
        print("Good bye!")
    except:
        print('something went wrong!')


         region  frequency
0       Seattle          2
1      Bellevue          0
2       Redmond          0
3      Kirkland          0
4       Bothell          0
..          ...        ...
120      Retsil          0
121      Olalla          0
122  Ravensdale          0
123   La Grande          0
124     Redondo          0

[125 rows x 2 columns]
          region  frequency
0        Seattle          2
6       Issaquah          1
18        Auburn          1
80    Longbranch          0
92        Conway          0
..           ...        ...
38    Silverdale          0
37    Marysville          0
36   Bonney Lake          0
35      Tumwater          0
124      Redondo          0

[125 rows x 2 columns]
enter 1 to calculate the count of sities showing up onthe crime news web page
enter 2 to show the data on console
enter 3 to clean the data
enter 0 to exit
